In [1]:
from qiskit import QuantumCircuit, converters
from qiskit import IBMQ, Aer, execute
from qiskit.providers.aer.noise import *
from qiskit.quantum_info import *
import qiskit.tools.jupyter
import random
import math
import numpy as np
# import basic plot tools
from qiskit.tools.visualization import *

In [2]:
provider = IBMQ.load_account()
real_backend = provider.get_backend('ibmq_armonk')
properties = real_backend.properties()    
coupling_map = real_backend.configuration().coupling_map
noise_model = NoiseModel.from_backend(properties)
basis_gates = noise_model.basis_gates

In [3]:
%qiskit_version_table

In [8]:
def create_random_circuit(num_qubits, depth=1, only_gates=None):    
    circ = QuantumCircuit(num_qubits, num_qubits)
    # set the starting state
    for d in range(depth):
        for q in range(num_qubits):
            add_gate = {
                'u3': lambda: circ.u3(random.uniform(0, math.pi), random.uniform(0, math.pi), random.uniform(0, math.pi), q),
                'x': lambda: circ.x(q),
                'y': lambda: circ.y(q),
                'z': lambda: circ.z(q),
                'h': lambda: circ.h(q)
            }

            if only_gates is None:
                add_gate.get('u3')()
            else:
                add_gate.get(only_gates[random.randrange(len(only_gates))])()
            
    return circ

In [14]:
def run(circ, shots, backend=None, noisy=False, disp=False):
    if (backend == None):
        backend = Aer.get_backend('qasm_simulator')
#         backend = Aer.get_backend('statevector_simulator')
    
    using_statevector_backend = backend.name() == 'statevector_simulator'
    if (noisy and not using_statevector_backend):
        print("Using noisy simulator")
        job = execute(circ, backend, shots=shots, coupling_map=coupling_map, noise_model=noise_model, basis_gates=basis_gates)
    else:
        job = execute(circ, backend, shots=shots)
    results = job.result()
    counts = results.get_counts()
    statevector = results.get_statevector(decimals=6) if using_statevector_backend else '[no statevector]'
    if disp:
        print("%s %s" % (counts, statevector))
        if using_statevector_backend:
            plot_bloch_multivector(statevector)
        else:
            plot_histogram(counts)
    return counts

def run_statevector(circ, disp=False):
    statevector_circ = circ.remove_final_measurements(inplace=False)
    backend = Aer.get_backend('statevector_simulator')
    job = execute(statevector_circ, backend, shots=1)
    results = job.result()
    statevector = results.get_statevector()
    if disp:
        print("%s" % (statevector))
        iplot_bloch_multivector(statevector)
    return statevector

def run_unitary(circ, disp=False):
    unitary_circ = circ.remove_final_measurements(inplace=False)
    backend = Aer.get_backend('unitary_simulator')
    job = execute(unitary_circ, backend, shots=1)
    results = job.result()
    unitary = results.get_unitary()
    if disp:
        print("%s" % (unitary))
    return unitary

In [128]:
def compare_statevector_fidelity(random_circ, circ):
    rsv = Statevector(run_statevector(random_circ))
    sv = Statevector(run_statevector(circ))
    rsv_counts = rsv.probabilities_dict()
    sv_counts = sv.probabilities_dict()
    return hellinger_fidelity(rsv_counts, sv_counts)

def compare_statevector_equiv(random_circ, circ):
    rsv = Statevector(run_statevector(random_circ))
    sv = Statevector(run_statevector(circ))
    return rsv.equiv(sv)

def compare_circuit_norm(random_circ, circ):
    return diamond_norm(Choi(random_circ) - Choi(circ), verbose=True)

def compare_circuit_unitaries(random_circ, circ):
    ru = run_unitary(random_circ)
    su = run_unitary(circ)
    return 1 - np.tanh(np.linalg.norm(ru - su, ord=2)) # convert L2 norm to a [0-1) value

def check_circuit(random_circ, circ, level=0):
    fidelity = [compare_statevector_fidelity(random_circ, circ)]
#     print(fidelity)
    
    if level > 0:        
        fidelity.append(compare_statevector_equiv(random_circ, circ))
#         print(fidelity)
    
    if level > 1:
        fidelity.append(compare_circuit_unitaries(random_circ, circ))
#         print(fidelity)
        
    if level > 2:
        fidelity.append(compare_circuit_norm(random_circ, circ))
#         print(fidelity)
    
    total_fidelity = np.mean(fidelity)
    
    if np.isclose(total_fidelity, 1.0):
        print("Congratulations, your intuition has succeeded. Here is the circuit:")
        print(random_circ)
    else:
        print("Not quite there yet: %s (fidelity)" % total_fidelity)

In [43]:
rc = create_random_circuit(1, depth=2, only_gates=['x', 'y', 'z'])

In [135]:
circ = QuantumCircuit(1, 1)
circ.x(0)
circ.y(0)
check_circuit(rc, circ, level=1)

Congratulations, your intuition has succeeded. Here is the circuit:
     ┌───┐┌───┐
q_0: ┤ X ├┤ Y ├
     └───┘└───┘
c_0: ══════════
               


In [131]:
rc.draw()

┌───┐┌───┐
q_0: ┤ X ├┤ Y ├
     └───┘└───┘
c_0: ══════════